In [1]:
import chromadb
from chromadb.utils.data_loaders import ImageLoader
import pickle
from sklearn.model_selection import train_test_split
import os

In [2]:
client = chromadb.PersistentClient(path= "/home/greatness-within/Downloads/animal_embs")
data_loader = ImageLoader()

In [3]:
collection = client.get_or_create_collection(name="animal_embs_new_10",data_loader = data_loader,
                                             metadata={"hnsw:space": "cosine"})

In [4]:
all_paths = []
for dir, dir_name, files in os.walk("/home/greatness-within/Downloads/animals/cropped/"):
    for file_name in files:
        path = os.path.join(dir, file_name)
        all_paths.append(path)

In [5]:
with open("/home/greatness-within/Downloads/animal_embs/names", 'rb') as file:
    # Deserialize and retrieve the variable from the file
    all_names = pickle.load(file)

with open("/home/greatness-within/Downloads/animal_embs/embeddings", 'rb') as file:
    # Deserialize and retrieve the variable from the file
    all_embeddings = pickle.load(file)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(all_embeddings,  all_names, test_size= 0.3, random_state=42)

In [7]:
collection.upsert(
    embeddings = all_embeddings[:len(X_train)],
    ids = [str(id ) for id in range(0,len(X_train) )],
    uris = all_paths[:len(X_train)],
    metadatas = [{"animals": all_names[i][:-3]} for i in range(len(X_train))]
)

In [11]:
i = 0
correct = 0
wrong = 0
other = 0
for embedding in X_test:
    result = collection.query(query_embeddings = embedding)
    # print(result["metadatas"][0][0]["animals"])
    if((result["metadatas"][0][0]["animals"] == y_test[i][:-3])):
        # print("predicted is ", result["metadatas"][0][0]["person"])
        # print("actual is    ", y_test[i])
        correct = correct + 1
        i = i+1
    elif(result["distances"][0][0] > 0.15 and (result["metadatas"][0][0]["animals"] != y_test[i])):
        # print("predicted is ", result["metadatas"][0][0]["person"])
        # print("actual is", y_test[i])
        other = other + 1
        i = i+1

    else:
        # print("something went wrong in ", y_test[i])
        wrong = wrong + 1
        i = i + 1


print("number of wrong predictions: ",wrong)
print("number of right predictions: ",correct)


number of wrong predictions:  2
number of right predictions:  84


In [9]:
result = collection.query(query_embeddings = X_train)

In [13]:
correct/(correct+wrong)

0.9767441860465116